In [1]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
import pickle

from keras import backend as K
import numpy as np
import string
from datetime import datetime 
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

W0520 20:46:49.698101 140341070206848 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
!pip install bert-tensorflow
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization


     |████████████████████████████████| 71kB 3.3MB/s 


In [3]:
# initialiase tensorboard 
# from https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab

# Get TensorBoard running in the background. 
LOG_DIR = './test_output'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# # #Download and unzip ngrok. 
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

# #Launch ngrok background process...
get_ipython().system_raw('./ngrok http 6006 &')


--2019-05-20 20:46:56--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.21.103.149, 34.231.75.48, 52.203.102.189, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.21.103.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  33.1MB/s    in 0.5s    

2019-05-20 20:46:57 (33.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [4]:

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://f48852f2.ngrok.io


In [5]:
import os, sys
sys.path.append('../') # add personal code dir to path for import


!test -d neural_process || git clone https://github.com/JTT94/nlp_neural_process.git neural_process
sys.path.append('./neural_process/')

import random
from neural_process import split_context_target, NeuralProcessParams
from neural_process.network import *
from neural_process.loss import *
from neural_process.predict import *
from neural_process.process import *

from neural_process.tf_bertNPmodel_builder import *
from neural_process.bert_utils import *
from neural_process.neural_process_utils import *

Cloning into 'neural_process'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 89 (delta 52), reused 47 (delta 24), pack-reused 0
Unpacking objects: 100% (89/89), done.


In [6]:
# Initialize session
sess = tf.Session()
K.set_session(sess)
K.tensorflow_backend._get_available_gpus()
# tf.logging.set_verbosity(tf.logging.ERROR)

['/job:localhost/replica:0/task:0/device:GPU:0']

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# filename = './cleaned_data.csv'
# filename = './data1.csv'
# df = pd.read_csv(filename, index_col=0)
# cols = ['comment','antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', 'healthy']
# cols = ['comment','cleaned_comment','antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', 'healthy']
# df = df[cols]

#score_column = ['antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', 'healthy']
# text_col_name = 'cleaned_comment'
# text_col_name = 'comment'

#--------

### For kaggle dataset

## Training data

# filename = './gdrive/My Drive/Data/train.csv'
train_filename = './gdrive/My Drive/Kaggle_toxic_comments/kaggle_train.csv'

df = pd.read_csv(train_filename)
cols = ['comment_text','toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df = df[cols]

score_column = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

text_col_name = 'comment_text'

#Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
for i in score_column:
  df[i] = pd.to_numeric(df[i],downcast='float')

# over represent toxic comments

df['num_toxic_atts'] = df[cols[1:]].apply(lambda x: np.sum(x), axis = 1)

df_toxic = df[df.num_toxic_atts  > 0]
# print(len(df_toxic))

df_healthy_sample = df[df.num_toxic_atts == 0].sample(frac=1.0)[:len(df_toxic)]
# print(len(df_healthy_sample))

df_train = pd.concat([df_toxic, df_healthy_sample]).sample(frac=1.0)


## Test data:

test_data_filename = './gdrive/My Drive/Kaggle_toxic_comments/kaggle_test.csv'
test_labels_filename = './gdrive/My Drive/Kaggle_toxic_comments/kaggle_test_labels.csv'
df_test = pd.read_csv(test_data_filename)
test_labels = pd.read_csv(test_labels_filename)
df_test = pd.merge(df_test, test_labels, on='id')
#get rid of -1 values
df_test = test_df.replace(-1, np.nan).dropna(subset=cols)

#restrict comment length
df_test = df_test[df_test.comment_text.str.len() <= 250]

df_test = df_test[cols]
#Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
for i in score_column:
  df_test[i] = pd.to_numeric(df_test[i],downcast='float')

In [0]:
# # Restrict comment length

# df = df[df.comment_text.str.len() <= 250]

# # print(len(df))

# # full_df = df.copy(deep=True)

# #sample portion of dataset to speed up preprocessing

# # df = df.sample(frac = 0.5)

# # over represent toxic comments

# df['num_toxic_atts'] = df[cols[1:]].apply(lambda x: np.sum(x), axis = 1)

# df_toxic = df[df.num_toxic_atts  > 0]
# # print(len(df_toxic))
# df_healthy_sample = df[df.num_toxic_atts == 0][:len(df_toxic)]
# print(len(df_healthy_sample))
# df_healthy_remaining = df[df.num_toxic_atts == 0][len(df_toxic):]
# print(len(df_healthy_remaining))

# df_overrep = pd.concat([df_toxic, df_healthy_sample]).sample(frac=1.0)
# print(len(df_overrep))
# train_propn = 0.8

# ratio = len(df_toxic)/(len(df_healthy_sample)+len(df_healthy_remaining))
# print(ratio)

# df_train = df_overrep[:int(len(df_overrep)*train_propn)]
# print(len(df_train))

# #construct a test set with similar proportion/imbalance in toxic - nontoxic data to the real data set
# df_test = pd.concat([df_overrep[int(len(df_overrep)*train_propn):], df_healthy_remaining[:int((1-0.8)*(len(df_healthy_remaining)-len(df_train)))]])

# print(0.5*len(df_overrep[int(len(df_overrep)*train_propn):]) / len(df_test))
# # df_train.head(50)

In [17]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
tokenizer = create_tokenizer_from_hub_module(BERT_model_hub)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0520 20:51:11.515383 140341070206848 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
# Parameters
max_seq_length = 128 # max number of tokens considered by bert per comment
num_draws = 20 
NPparams = NeuralProcessParams(dim_r=20, dim_z=20, n_hidden_units_h=[128, 128, 128], n_hidden_units_g=[128, 128, 128])


In [0]:
# #Pre process data for bert embedding

# train_input_examples = create_examples(df_train, score_column, text_col_name)
# test_input_examples = create_examples(df_test, score_column, text_col_name)

# train_features = convert_examples_to_features(train_input_examples, max_seq_length, tokenizer)
# test_features = convert_examples_to_features(test_input_examples, max_seq_length, tokenizer)

# pickle.dump(train_features, open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'wb'))
# pickle.dump(test_features, open('./gdrive/My Drive/Kaggle_toxic_comments/test_features.p', 'wb'))

# # ## Load features previously saved

train_features = pickle.load(open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'rb'))
test_features = pickle.load(open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'rb'))

In [0]:
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
batch_size = 32
# lr = 2e-2
lr = 2e-5
epochs = 15.0
# Warmup is a period of time where the learning rate  is small and gradually increases
warmpup_proportion = 0.1

# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / batch_size *epochs)
num_warmup_steps = int(num_train_steps * warmpup_proportion)

#####

output_dir = "./test_output"
save_checkpoints_steps = 500
save_summary_steps = 100

# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(model_dir=output_dir,
    save_summary_steps=save_summary_steps, save_checkpoints_steps=save_checkpoints_steps)

#####

num_labels = len(score_column)

model_fn = model_fn_builder(create_model, num_labels = num_labels, learning_rate=lr,
  NPparams = NPparams, num_draws=num_draws , BERT_model_hub = BERT_model_hub, num_train_steps=num_train_steps, num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(model_fn=model_fn,config=run_config,
  params={"batch_size": batch_size})

#####

# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = input_fn_builder(
    features=train_features, seq_length=max_seq_length, 
    num_labels = num_labels, is_training=True, drop_remainder=False)

#####

print('Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

INFO:tensorflow:Using config: {'_model_dir': './test_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa303d997f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0521 00:00:27.040299 140341070206848 estimator.py:201] Using config: {'_model_dir': './test_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa303d997f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Beginning Training!
INFO:tensorflow:Calling model_fn.


I0521 00:00:45.173328 140341070206848 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

I0521 01:06:20.092262 140341070206848 basic_session_run_hooks.py:247] loss = -101.58249, step = 14440 (87.761 sec)


INFO:tensorflow:global_step/sec: 1.13939


I0521 01:07:47.855243 140341070206848 basic_session_run_hooks.py:680] global_step/sec: 1.13939


INFO:tensorflow:loss = -122.964714, step = 14540 (87.766 sec)


I0521 01:07:47.858685 140341070206848 basic_session_run_hooks.py:247] loss = -122.964714, step = 14540 (87.766 sec)


In [0]:
# ! cp -r ./test_output "./gdrive/My Drive/Data/"

In [0]:
eval_steps = int(len(df_test) / batch_size)
eval_input_fn = input_fn_builder(
    features=test_features,
    seq_length=max_seq_length,
    num_labels = num_labels,
    is_training=False,
    drop_remainder=True)
result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

In [0]:
print("***** Eval results *****")
for key in sorted(result.keys()):
  print('  {} = {}'.format(key, str(result[key])))
      

In [0]:
predict_features = test_features[:100]

In [0]:
test_input_fn = input_fn_builder(
    features=predict_features, seq_length=max_seq_length, 
    num_labels = num_labels, is_training=False, drop_remainder=False)

In [0]:
test_scores = np.array([predict_features[i].score for i, val in enumerate(predict_features)])

preds = estimator.predict(input_fn=test_input_fn)
preds_vals = list(preds)
pred_means = np.array([val['prediction_mean'] for key, val in enumerate(preds_vals)])

INFO:tensorflow:Calling model_fn.


I0520 15:50:48.501365 140104395057024 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0520 15:50:53.082599 140104395057024 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Tensor("module_apply_tokens/bert/pooler/dense/Tanh:0", shape=(?, 768), dtype=float32)


...

I0520 15:51:09.580962 140104395057024 saver.py:1270] Restoring parameters from ./test_output/model.ckpt-3492


INFO:tensorflow:Running local_init_op.


I0520 15:51:12.383657 140104395057024 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0520 15:51:12.701397 140104395057024 session_manager.py:493] Done running local_init_op.


In [0]:
def total_accuaracy(pred_means, test_scores):
  preds = np.round(pred_means)
  correct_guesses = np.reshape(preds == test_scores,-1)
  return np.sum(correct_guesses) / len(correct_guesses)

In [0]:
def class_accuaracy(pred_means, test_scores):
  preds = np.round(pred_means)
  correct_guesses = preds == test_scores
  return correct_guesses.sum(axis=0)/ len(correct_guesses)


In [0]:
print(total_accuaracy(pred_means, test_scores))
print(class_accuaracy(pred_means, test_scores))

0.93
[0.94 0.97 0.94 0.96 0.81 0.96]


In [0]:
preds_vals

[{'prediction_mean': array([0.99396497, 0.19381616, 0.97161853, 0.01611928, 0.9024913 ,
         0.09952925], dtype=float32),
  'prediction_var': array([3.3862303e-08, 1.7771668e-05, 3.0750897e-07, 2.3864851e-07,
         1.3581971e-06, 6.0153079e-06], dtype=float32)},
 {'prediction_mean': array([0.9612483 , 0.03642098, 0.2432679 , 0.02208908, 0.70222914,
         0.04978191], dtype=float32),
  'prediction_var': array([1.5239682e-06, 5.5279327e-07, 1.7787603e-05, 3.0437258e-07,
         1.8960136e-05, 1.0327615e-06], dtype=float32)},
 {'prediction_mean': array([0.9710032 , 0.00946731, 0.24418692, 0.00585204, 0.1644721 ,
         0.0159474 ], dtype=float32),
  'prediction_var': array([9.8179612e-07, 4.4951655e-08, 3.3007473e-05, 2.1189376e-08,
         1.0118373e-05, 1.2566389e-07], dtype=float32)},
 {'prediction_mean': array([0.95585966, 0.02739453, 0.17853627, 0.01882137, 0.625805  ,
         0.03973923], dtype=float32),
  'prediction_var': array([1.9052101e-06, 3.2246538e-07, 1.19922

In [0]:
test_scores

array([[1., 0., 1., 0., 0., 0.],
       [1., 0., 1., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1.